In [46]:
import scipy.io as scio
import pandas as pd
import numpy as np
import os
from pandas import Series,DataFrame

from sklearn.metrics import accuracy_score  

import time

### Set Path 

In [47]:
pwd = os.path.dirname(os.path.dirname(os.getcwd()))
checkpoint_wd = os.path.join(pwd, "output", "checkpoint")
pwd

'/Users/yuting/Desktop/MS&E_3/Applied DS/Fall2019-proj3-sec1--proj3-sec1-grp2'

### Read Saved Model and Test Data 

In [48]:
# read model
baseline_model = joblib.load(checkpoint_wd+"/baseline_model.m")
improved_model = joblib.load(checkpoint_wd+"/improved_model.m")

In [49]:
# read label
data_path = os.path.join(pwd, "data", "test_set")
df = pd.read_csv(data_path +"/label.csv")
labels = df["emotion_idx"].tolist()

In [50]:
scaler = MinMaxScaler(feature_range = (0,1))

In [51]:
def distance_feature(x): # calcuate distance between points, choose uprigt triangle
    #sqrt(dx^2+dy^2)
    a = (x[None, :, :] - x[:, None, :])
    b = np.linalg.norm(a, axis=2)
    return b[np.triu_indices(78, 1)] # feature size (78-1)(78-2) /2 

In [52]:
# read point
data = list()
for i in range(1, 2501):
    name = "../../data/test_set/points/%04d.mat" % i
    dic = scio.loadmat(name)
    if "faceCoordinatesUnwarped" in dic:
        data.append(dic["faceCoordinatesUnwarped"])
    else:
        data.append(dic["faceCoordinates2"])

### Normalization 

In [53]:
#Normalization
old_train = np.load("../../output/train_x.npy")
test_x = [distance_feature(d) for d in data]
scaler.fit(old_train)
test_x = scaler.transform(test_x)

### Test Result 

Baseline Model

In [56]:
baseline_preds = baseline_model.predict(test_x)

In [57]:
test_accuracy = accuracy_score(labels, baseline_preds) 
print ("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))

Test Accuracy: 39.56%


Improved Model

In [58]:
improved_preds = improved_model.predict(test_x)
test_accuracy = accuracy_score(labels, improved_preds) 
print ("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))

Test Accuracy: 72.00%


### Output 

In [ ]:
baseline_preds.to_csv('../../output/Baseline_Result.csv')

In [ ]:
improved_preds.to_csv('../../output/Improved_Result.csv')